# pyHGT Model Training

In [7]:
import os
import numpy as np
import pandas as pd
from collections import defaultdict

In [2]:
import sys
print(sys.executable)

c:\Users\Anushka\miniconda3\envs\pyhgt38\python.exe


In [3]:
import matplotlib
print(matplotlib.__version__)
print(matplotlib.__file__)

3.1.3
c:\Users\Anushka\miniconda3\envs\pyhgt38\lib\site-packages\matplotlib\__init__.py


In [4]:
import seaborn, inspect
print(seaborn.__version__)
print(seaborn.__file__)

0.9.0
c:\Users\Anushka\miniconda3\envs\pyhgt38\lib\site-packages\seaborn\__init__.py


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

%cd ../
from pyHGT.data import Graph, sample_subgraph, to_torch
from pyHGT.model import GNN, Classifier

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

c:\Users\Anushka\miniconda3\envs\pyhgt38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


d:\FYP\CODE\pyHGT-implementation
Using device: cuda


In [9]:
# Load the data files

data_dir = 'data/'

patient_tests = pd.read_csv(os.path.join(data_dir,"patient-test.csv"), encoding='latin1')
test_details = pd.read_csv(os.path.join(data_dir,"test-disease-organ.csv"), encoding='latin1')
labels_df = pd.read_csv(os.path.join(data_dir,"patient-diseases.csv"), encoding='latin1')

In [10]:
display(patient_tests.head())

,patient_id,report_date,test_name,test_value
0,139760,9/2/2024 6:13,Blood Urea Result,66.0
1,139760,9/2/2024 6:14,ASOT Result,200.0
2,139760,9/2/2024 6:15,ALP Result,168.0
3,200041,10/7/2024 20:59,Fasting Plasma Glucose,83.0
4,200041,10/7/2024 20:59,1 Hr After Plasma Glucose,83.0


In [11]:
display(test_details.head())

,test_name,min,max,organ,organ.1,disease,disease.1,disease.2
0,TSH,0.40,4.00,thyroid,NaN,thyroid disorders,NaN,NaN
1,Fasting Venous Plasma Glucose,100.00,126.00,pancreas,NaN,diabetes mellitus,NaN,NaN
2,Lymphocytes# % Value,20.00,40.00,immune system,NaN,infections,Immune disorders,NaN
3,RDW-SD Absolute Value,39.00,46.00,blood,NaN,anemia,NaN,NaN
4,RBC Absolute Value,4.11,5.51,blood,NaN,anemia,NaN,NaN


In [13]:
display(labels_df.head())

,patient_id,diseases
0,139760,biliary obstruction; chronic kidney disease
1,200041,diabetes mellitus
2,201519,NaN
3,201605,leukemia; acute; infections; kidney disease; c...
4,201839,NaN


In [31]:
patient_tests["patient_id"] = patient_tests["patient_id"].astype(str)
patient_tests["test_name"] = patient_tests["test_name"].astype(str)

test_details["test_name"] = test_details["test_name"].astype(str)

if "organ" in test_details.columns:
    test_details["organ"] = test_details["organ"].astype(str)

if "disease" in test_details.columns:
    test_details["disease"] = test_details["disease"].astype(str)

test_details = test_details.drop_duplicates(subset=["test_name"]).reset_index(drop=True)

#temporal processing
patient_tests["report_date"] = pd.to_datetime(patient_tests["report_date"])
patient_tests["time_idx"] = patient_tests["report_date"].astype("int64") // 10**9  # convert to unix timestamp in seconds

min_time = patient_tests["time_idx"].min()
patient_tests["rel_time"] = (patient_tests["time_idx"]-min_time)// 86400  # convert to days

In [33]:
display(patient_tests.head())

,patient_id,report_date,test_name,test_value,time_idx,rel_time
0,139760,2024-09-02 06:13:00,Blood Urea Result,66.0,1725257580,0
1,139760,2024-09-02 06:14:00,ASOT Result,200.0,1725257640,0
2,139760,2024-09-02 06:15:00,ALP Result,168.0,1725257700,0
3,200041,2024-10-07 20:59:00,Fasting Plasma Glucose,83.0,1728334740,36
4,200041,2024-10-07 20:59:00,1 Hr After Plasma Glucose,83.0,1728334740,36


In [34]:
def parse_multi(x):
    if pd.isna(x):
        return []
    x = str(x).strip()

    if ";" in x:
        return [item.strip() for item in x.split(";")]
    
    if "," in x:
        return [item.strip() for item in x.split(",")]
    
    return [x] if x else []


lab_info = {}

for _, row in test_details.iterrows():
    test = str(row["test_name"]).strip()

    organs = parse_multi(row.get("organ", ""))
    diseases = parse_multi(row.get("disease", ""))

    low_th = None
    high_th = None
    if "min" in row and not pd.isna(row["min"]):
        low_th = float(row["min"])
    if "max" in row and not pd.isna(row["max"]):
        high_th = float(row["max"])

    lab_info[test] = {
        "organs": organs,
        "diseases": diseases,
        "low": low_th,
        "high": high_th
    }

print("example lab info:", list(lab_info.items())[:3]) # only show 3 examples

example lab info: [('TSH', {'organs': ['thyroid'], 'diseases': ['thyroid disorders'], 'low': 0.4, 'high': 4.0}), ('Fasting Venous Plasma Glucose', {'organs': ['pancreas'], 'diseases': ['diabetes mellitus'], 'low': 100.0, 'high': 126.0}), ('Lymphocytes#  % Value', {'organs': ['immune system'], 'diseases': ['infections'], 'low': 20.0, 'high': 40.0})]


In [37]:
pt_merged = patient_tests.merge(
    test_details[["test_name", "min", "max"]],
    on="test_name",
    how="left"
)

def compute_abnoramability(row):
    v = float(row["test_value"])
    low = row["min"]
    high = row["max"]

    if pd.isna(low) or pd.isna(high) or low >= high:
        return 0.0  # cannot determine abnormality
    return float((v - low) / (high - low))


pt_merged["abnormality"] = pt_merged.apply(compute_abnoramability, axis=1)

# aggregate per person
agg = pt_merged.groupby("patient_id").agg(
    num_tests = ("test_name", "count"),
    mean_abn = ("abnormality", "mean"),
    max_abn = ("abnormality", "max"),
    min_abn = ("abnormality", "min"),
    last_time = ("rel_time", "max")
).reset_index()

patient_feat_df = agg.set_index("patient_id")
patient_feat_df.head()

,num_tests,mean_abn,max_abn,min_abn,last_time
patient_id,,,,,
1001261,1,0.154722,0.154722,0.154722,131
1001280,3,0.265806,0.477000,0.000000,131
1001398,2,-0.970000,-0.100000,-1.840000,131
1001467,3,0.293917,0.707500,-0.207000,131
1001490,1,3.268182,3.268182,3.268182,132


In [38]:
# extract unique node ids
patient_ids = sorted(patient_tests["patient_id"].astype(str).unique().tolist())
lab_tests = sorted(list(lab_info.keys()))

all_organs = sorted({org for info in lab_info.values() for org in info["organs"] if org})
all_diseases = sorted({dis for info in lab_info.values() for dis in info["diseases"] if dis})

print("#patients =", len(patient_ids))
print("#labs =", len(lab_tests))
print("#organs =", len(all_organs))
print("#diseases =", len(all_diseases))

print("#nodes = ", len(patient_ids) + len(lab_tests) + len(all_organs) + len(all_diseases))

#patients = 24352
#labs = 172
#organs = 19
#diseases = 45
#nodes =  24588


In [39]:
#build pyHGT graph - add nodes

graph = Graph()

for pid in patient_ids:
    if pid in patient_feat_df.index:
        row = patient_feat_df.loc[pid]
        node = {
            "type": "patient",
            "id": pid,
            "num_tests": float(row["num_tests"]),
            "mean_abn":  float(row["mean_abn"]),
            "max_abn":   float(row["max_abn"]),
            "min_abn":   float(row["min_abn"]),
            "last_time": int(row["last_time"]),
            "time":      int(row["last_time"]),
        }
    
    else:
        node = {
            "type": "patient",
            "id": pid,
            "num_tests": 0.0,
            "mean_abn":  0.0,
            "max_abn":   0.0,
            "min_abn":   0.0,
            "last_time": 0,
            "time":      0,
        }
    graph.add_node(node)

for test in lab_tests:
    info = lab_info[test]
    graph.add_node({
        "type": "lab_test",
        "id": test,
        "time": 0,
        "low": 0.0 if info["low"] is None else float(info["low"]),
        "high": 0.0 if info["high"] is None else float(info["high"])
    })


for organ in all_organs:
    graph.add_node({
        "type": "organ",
        "id": organ,
        "time": 0
    })

for disease in all_diseases:
    graph.add_node({
        "type": "disease",
        "id": disease,
        "time": 0
    })

In [41]:
# Convenience maps from original ID -> internal index
patient2idx = graph.node_forward["patient"]
lab2idx     = graph.node_forward["lab_test"]
organ2idx   = graph.node_forward["organ"]
disease2idx = graph.node_forward["disease"]

print("Example patient2idx:", list(lab2idx.items())[:5])

Example patient2idx: [('1 Hr After Plasma Glucose', 0), ('2 Hr After Breakfast Plasma Glucose', 1), ('2 Hrs After Dinner Plasma Glucose', 2), ('2 Hrs After LunchPlasma Glucose', 3), ('2 Hrs After Plasma Glucose', 4)]


In [43]:
#build pyHGT graph - add edges with temporal info

for _, row in patient_tests.iterrows():
    pid = str(row["patient_id"])
    test = str(row["test_name"])
    t = int(row["rel_time"])

    if pid in patient2idx and test in lab2idx: # src and dst nodes exist
        # patient - lab_test edge
        graph.add_edge(
            {"type": "patient", "id": pid},
            {"type": "lab_test", "id": test},
            time=t,
            relation_type="had_test",
            directed=True,
        )

for test, info in lab_info.items():

    # lab_test - organ edges
    for organ in info["organs"]:
        if organ in organ2idx:
            graph.add_edge(
                {"type": "lab_test", "id": test},
                {"type": "organ", "id": organ},
                relation_type="tests_organ",
                directed=True,
                time=0
            )

    # lab_test - disease edges
    for disease in info["diseases"]:
        if disease in disease2idx:
            graph.add_edge(
                {"type": "lab_test", "id": test},
                {"type": "disease", "id": disease},
                relation_type="associated_with",
                directed=True,
                time=0
            )

            for organ in info["organs"]:
                if organ in organ2idx:
                    graph.add_edge(
                        {"type": "disease", "id": disease},
                        {"type": "organ", "id": organ},
                        relation_type="occurs_in",
                        directed=True,
                        time=0
                    )

print("Meta relations in graph:", graph.get_meta_graph())

Meta relations in graph: [('lab_test', 'patient', 'had_test'), ('lab_test', 'organ', 'rev_tests_organ'), ('lab_test', 'disease', 'rev_associated_with'), ('patient', 'lab_test', 'rev_had_test'), ('organ', 'lab_test', 'tests_organ'), ('organ', 'disease', 'occurs_in'), ('disease', 'lab_test', 'associated_with'), ('disease', 'organ', 'rev_occurs_in')]


In [44]:
#node features

for t, node_list in graph.node_bacward.items():
    df = pd.DataFrame(node_list).reset_index(drop=True)
    graph.node_feature[t] = df

for t,df in graph.node_feature.items():
    print(f"Node type: {t}, feature shape: {df.shape}")
    display(df.head())

Node type: patient, feature shape: (24352, 8)


,type,id,num_tests,mean_abn,max_abn,min_abn,last_time,time
0,patient,1001261,1.0,0.154722,0.154722,0.154722,131,131
1,patient,1001280,3.0,0.265806,0.477000,0.000000,131,131
2,patient,1001398,2.0,-0.970000,-0.100000,-1.840000,131,131
3,patient,1001467,3.0,0.293917,0.707500,-0.207000,131,131
4,patient,1001490,1.0,3.268182,3.268182,3.268182,132,132


Node type: lab_test, feature shape: (172, 5)


,type,id,time,low,high
0,lab_test,1 Hr After Plasma Glucose,0,0.0,180.0
1,lab_test,2 Hr After Breakfast Plasma Glucose,0,0.0,140.0
2,lab_test,2 Hrs After Dinner Plasma Glucose,0,0.0,140.0
3,lab_test,2 Hrs After LunchPlasma Glucose,0,0.0,140.0
4,lab_test,2 Hrs After Plasma Glucose,0,0.0,140.0


Node type: organ, feature shape: (19, 3)


,type,id,time
0,organ,Immune system,0
1,organ,Kidney,0
2,organ,blood,0
3,organ,bone narrow,0
4,organ,cardiovascular system,0


Node type: disease, feature shape: (45, 3)


,type,id,time
0,disease,acute,0
1,disease,adrenal gland disorders,0
2,disease,allergies,0
3,disease,anemia,0
4,disease,anemia (B12/folate),0


In [52]:
# feature_medical extractor for sample subgraph

def feature_medical(layer_data, graph):
    feature = {}
    times = {}
    indxs = {}
    texts = {}

    # for t in layer_data:
    #     if len(layer_data[t]) == 0:
    #         continue

    all_types = graph.node_feature.keys()

    for t in all_types:
        if t not in layer_data or len(layer_data[t]) == 0:
            feature[t] = np.zeros((0,5), dtype=np.float32)  # empty features
            times[t] = np.array((0,), dtype=np.int32)
            indxs[t] = np.array((0,), dtype=np.int32)
            continue

        idxs = np.array(list(layer_data[t].keys()))
        tims = np.array(list(layer_data[t].values()))[:,1]

        df = graph.node_feature[t]

        feats = np.zeros((len(idxs),5), dtype=np.float32)  # 5 features per node

        if t == "patient":

            cols = ["num_tests", "mean_abn", "max_abn", "min_abn", "last_time"]
            for c in cols:
                if c not in df.columns:
                    df[c] = 0.0

            vals = df.loc[idxs, cols].fillna(0).values.astype(np.float32)
            feats = vals # all 5 features

        elif t == "lab_test":
            for c in ["low", "high"]:
                if c not in df.columns:
                    df[c] = 0.0
            vals = df.loc[idxs, ["low", "high"]].fillna(0).values.astype(np.float32)
            feats[:,0:2] = vals  # first 2 features and other 3 are zeros

        else:
            pass  # no features for organ and disease nodes

        feature[t] = feats
        times[t] = tims
        indxs[t] = idxs

    return feature, times, indxs, texts    

In [54]:
#subgraph sampling test

from pyHGT.data import sample_subgraph, to_torch

all_patient_indices = np.array(list(graph.node_forward["patient"].values()))
batch_pids = all_patient_indices[:8] # first 8 patients for testing

inp = {
    "patient": [(int(pid),0) for pid in batch_pids]
}

feature, times, edge_list, indxs, texts = sample_subgraph(
    graph,
    time_range={0: True},
    sampled_depth=2,
    sampled_number=8,
    inp = inp,
    feature_extractor=feature_medical
)

node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = to_torch(
    feature,times,edge_list, graph
)

In [55]:
print("node_feature:", node_feature.shape)
print("node_type:", node_type.shape)
print("edge_index:", edge_index.shape)
print("edge_time:", edge_time.shape)
print("edge_type:", edge_type.shape)
print("node_dict:", node_dict)
print("edge_dict:", edge_dict)

node_feature: torch.Size([8, 5])
node_type: torch.Size([8])
edge_index: torch.Size([2, 8])
edge_time: torch.Size([8])
edge_type: torch.Size([8])
node_dict: {'patient': [0, 0], 'lab_test': [8, 1], 'organ': [8, 2], 'disease': [8, 3]}
edge_dict: {'had_test': 0, 'rev_tests_organ': 1, 'rev_associated_with': 2, 'rev_had_test': 3, 'tests_organ': 4, 'occurs_in': 5, 'associated_with': 6, 'rev_occurs_in': 7, 'self': 8}
